In [28]:
import pandas as pd
import json
import nltk

In [3]:
with open('speeches.json') as f:
  test = json.load(f)

## Extract speech from JSON

In [25]:
def create_bow(json):
    df=pd.DataFrame(columns=['id','speaker','date','speech'])
    for i in range(len(json)):
        df.at[i,'id'] = json[i]['items'][0]['id']
        df.at[i,'speaker'] = json[i]['items'][0]['candidate']
        df.at[i,'date'] = json[i]['items'][0]['snippet']['publishedAt']
        captions = test[i]['items'][0]['captions']
        just_text = [j['text'] for j in captions]
        speech = ' '.join(just_text)
        df.at[i,'speech'] = speech
    return df

## Tokenize

In [35]:
def tokenize(df, remove_pos_tuple=False):

    df = df.speech\
        .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
        .stack()\
        .to_frame()\
        .rename(columns={0:'pos_tuple'})
    
    # Grab info from tuple
    df['pos'] = df.pos_tuple.apply(lambda x: x[1])
    df['token_str'] = df.pos_tuple.apply(lambda x: x[0])
    if remove_pos_tuple:
        df = df.drop('pos_tuple', 1)
        
    return df

In [37]:
TOKEN = tokenize(check)

In [38]:
TOKEN['term_str'] = TOKEN['token_str'].str.lower().str.replace('[\W_]', '')

## Create VOCAB table

In [39]:
VOCAB = TOKEN.term_str.value_counts().to_frame().rename(columns={'index':'term_str', 'term_str':'n'})\
    .sort_index().reset_index().rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_id'

## Add stopwords

In [43]:
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1
VOCAB['stop'] = VOCAB.term_str.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')

In [44]:
VOCAB

,term_str,n,stop
term_id,,,
0,000,4,0
1,1,2,0
2,10,3,0
3,100,1,0
4,104,1,0
...,...,...,...
1980,youve,10,0
1981,zach,2,0
1982,zero,1,0
